In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.3.2`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._

val spark = SparkSession.builder()
  .appName("SparkTest")
  .master("local[*]")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark is ready!")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark is ready!


import $ivy.$                                      

import org.apache.spark.sql._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions._


spark: SparkSession = org.apache.spark.sql.SparkSession@3c847f

In [2]:
import spark.implicits._

case class Insurance(pid: Int, tiv_2015: Double, tiv_2016: Double, lat: Double, lon: Double)

val insurance: Seq[(Int, Double, Double, Double, Double)] = Seq(
  (1, 10.0, 5.0, 10.0, 10.0),
  (2, 20.0, 20.0, 20.0, 20.0),
  (3, 10.0, 30.0, 20.0, 20.0),
  (4, 10.0, 40.0, 40.0, 40.0)
)

val insuranceDS = insurance.toDF("pid", "tiv_2015", "tiv_2016", "lat", "lon").as[Insurance]

import spark.implicits._


defined class Insurance
insurance: Seq[(Int, Double, Double, Double, Double)] = List(
  (1, 10.0, 5.0, 10.0, 10.0),
  (2, 20.0, 20.0, 20.0, 20.0),
  (3, 10.0, 30.0, 20.0, 20.0),
  (4, 10.0, 40.0, 40.0, 40.0)
)
insuranceDS: Dataset[Insurance] = [pid: int, tiv_2015: double ... 3 more fields]

Write a solution to report the sum of all total investment values in 2016 tiv_2016, for all policyholders who:

* have the same tiv_2015 value as one or more other policyholders, and
* are not located in the same city as any other policyholder (i.e., the (lat, lon) attribute pairs must be unique).

Round tiv_2016 to two decimal places.

The result format is in the following example.

| tiv_2016 |
|----------|
| 45.00    |

In [3]:
val valid2015DF = insuranceDS.groupBy(col("tiv_2015"))
    .agg(count(lit(1)).as("cnt"))
    .filter(col("cnt") > 1)
    .drop("cnt")

val invalidLocDF = insuranceDS.groupBy(col("lat"), col("lon"))
    .agg(count(lit(1)).as("cnt"))
    .filter(col("cnt") > 1)
    .drop(col("cnt"))

insuranceDS.join(valid2015DF, Seq("tiv_2015"), "inner")
    .join(invalidLocDF, Seq("lat", "lon"), "anti")
    .agg(round(sum("tiv_2016"), 2).as("tiv_2016"))
    .show()


+--------+
|tiv_2016|
+--------+
|    45.0|
+--------+



valid2015DF: DataFrame = [tiv_2015: double]
invalidLocDF: DataFrame = [lat: double, lon: double]

In [4]:
spark.stop()